In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import distributed, sys, joblib
import mhcflurry, mhcflurry_cloud
import joblib.parallel
from joblib import Parallel
from joblib.parallel import register_parallel_backend, parallel_backend
from distributed.joblib import DistributedBackend
register_parallel_backend('distributed', DistributedBackend)


Using TensorFlow backend.


In [2]:
#executor = distributed.Executor('127.0.0.1:8786')

In [3]:
# Load some data.
full_train_data = mhcflurry.dataset.Dataset.from_csv("../test/data/bdata.2009.mhci.public.1.txt")
train_data = full_train_data.filter_alleles_by_count(5000)

# Generate cross validation folds, with imputation.
#imputer = fancyimpute.MICE(n_imputations=2, n_burn_in=1, n_nearest_columns=25)


folds = mhcflurry_cloud.cross_validation_folds(
    train_data,
    n_folds=3,
    imputer=None,
    drop_similar_peptides=True,
    #alleles=["HLA-A0201", "HLA-A0202"],
    n_jobs=-1,
    verbose=5,
)
folds

root - INFO - Allele: HLA-A0201
root - INFO - After dropping similar peptides, test size 3189->1525
root - INFO - After dropping similar peptides, test size 3188->1539
root - INFO - After dropping similar peptides, test size 3188->1423
root - INFO - Allele: HLA-A0301
root - INFO - After dropping similar peptides, test size 2047->1134
root - INFO - After dropping similar peptides, test size 2047->1073
root - INFO - After dropping similar peptides, test size 2047->1084
root - INFO - Allele: HLA-A1101
root - INFO - After dropping similar peptides, test size 1800->1019
root - INFO - After dropping similar peptides, test size 1800->998
root - INFO - After dropping similar peptides, test size 1799->982
root - INFO - Allele: HLA-A0203
root - INFO - After dropping similar peptides, test size 1848->980
root - INFO - After dropping similar peptides, test size 1847->955
root - INFO - After dropping similar peptides, test size 1847->984


[AlleleSpecificTrainTestFold(allele='HLA-A0201', train=Dataset(n=6376, alleles=['HLA-A0201']), imputed_train=None, test=Dataset(n=1525, alleles=['HLA-A0201'])),
 AlleleSpecificTrainTestFold(allele='HLA-A0201', train=Dataset(n=6377, alleles=['HLA-A0201']), imputed_train=None, test=Dataset(n=1539, alleles=['HLA-A0201'])),
 AlleleSpecificTrainTestFold(allele='HLA-A0201', train=Dataset(n=6377, alleles=['HLA-A0201']), imputed_train=None, test=Dataset(n=1423, alleles=['HLA-A0201'])),
 AlleleSpecificTrainTestFold(allele='HLA-A0301', train=Dataset(n=4094, alleles=['HLA-A0301']), imputed_train=None, test=Dataset(n=1134, alleles=['HLA-A0301'])),
 AlleleSpecificTrainTestFold(allele='HLA-A0301', train=Dataset(n=4094, alleles=['HLA-A0301']), imputed_train=None, test=Dataset(n=1073, alleles=['HLA-A0301'])),
 AlleleSpecificTrainTestFold(allele='HLA-A0301', train=Dataset(n=4094, alleles=['HLA-A0301']), imputed_train=None, test=Dataset(n=1084, alleles=['HLA-A0301'])),
 AlleleSpecificTrainTestFold(allel

In [4]:
# Make some models.
models = mhcflurry_cloud.models_grid(
    activation=["tanh", "relu"],
    layer_sizes=[[4]],
    embedding_output_dim=[8],
    n_training_epochs=[3])
models

[{'activation': 'tanh',
  'batch_size': 128,
  'dropout_probability': 0.5,
  'embedding_output_dim': 8,
  'fraction_negative': 0.2,
  'impute': False,
  'layer_sizes': [4],
  'max_ic50': 50000,
  'n_training_epochs': 3,
  'pretrain_decay': '1 / (1+epoch)**2'},
 {'activation': 'relu',
  'batch_size': 128,
  'dropout_probability': 0.5,
  'embedding_output_dim': 8,
  'fraction_negative': 0.2,
  'impute': False,
  'layer_sizes': [4],
  'max_ic50': 50000,
  'n_training_epochs': 3,
  'pretrain_decay': '1 / (1+epoch)**2'}]

In [12]:
with joblib.parallel_backend('distributed', scheduler_host='127.0.0.1:8786'):
    df = mhcflurry_cloud.train_across_models_and_folds(folds, models)
df


root - INFO - Training 2 architectures on 12 folds = 24 predictors.
root - INFO - Done.


,allele,fold_num,model_num,train_size,test_size,train_auc,train_f1,train_tau,test_auc,test_f1,...,model_impute,model_activation,model_layer_sizes,model_batch_size,model_fraction_negative,model_n_training_epochs,model_embedding_output_dim,model_pretrain_decay,model_dropout_probability,model_max_ic50
0,HLA-A0201,0,0,6376,1525,0.895284,0.587408,0.517057,0.892627,0.540448,...,False,tanh,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000
1,HLA-A0201,0,1,6376,1525,0.826558,0.099691,0.413170,0.819003,0.043796,...,False,relu,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000
2,HLA-A0201,1,0,6377,1539,0.885063,0.492090,0.496178,0.894675,0.408759,...,False,tanh,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000
3,HLA-A0201,1,1,6377,1539,0.849540,0.398695,0.445013,0.850579,0.366300,...,False,relu,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000
4,HLA-A0201,2,0,6377,1423,0.871091,0.513023,0.467363,0.874031,0.472998,...,False,tanh,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000
5,HLA-A0201,2,1,6377,1423,0.855029,0.250103,0.440961,0.863000,0.226328,...,False,relu,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000
6,HLA-A0301,3,0,4094,1134,0.800724,0.460327,0.352416,0.789823,0.474725,...,False,tanh,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000
7,HLA-A0301,3,1,4094,1134,0.705915,0.000000,0.249910,0.698282,0.000000,...,False,relu,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000
8,HLA-A0301,4,0,4094,1073,0.814982,0.495316,0.390080,0.844880,0.430380,...,False,tanh,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000
9,HLA-A0301,4,1,4094,1073,0.763060,0.192000,0.320746,0.755124,0.210863,...,False,relu,[4],128,0.2,3,8,1 / (1+epoch)**2,0.5,50000


In [6]:
df

""
